# Formula 1 data analysis application season 2023
This is the 2023 version

This time, I start from what I did last year and try to have a better data visualisation

I will try to separate the battles

## Imports

In [1]:
import json

import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from ipywidgets import widgets

## General Part
In this part, I will be interested on the general visualisation of the season
It will stay close to what I did last year

### Backend

In [2]:
# Import data
data_all = pd.read_json('data_2023.json')
data_all["index"] = data_all.groupby("pilot").cumcount()
data_all["index"] += 1
data_all["improvement"] = data_all["grid"] - data_all["result"]

# Dataset of the cumulative points of each pilot
data_points = data_all[["pilot", "points", "circuit_name"]].copy()
data_points["cum_points"] = data_points.groupby("pilot").cumsum()
data_points["index"] = data_points.groupby("pilot").cumcount()

# Extraction of pilots per team
with open("teams_2023.json") as jsonfile:
    teams_pilot = json.load(jsonfile)

data_team_points = {"team": [], "points": [], "cum_points": [], "index": [], "circuit_name": []}
for i in data_points["index"].unique():
    for team in teams_pilot:
        data_team_points["team"].append(team)
        data_team_points["points"].append(data_points[data_points["index"] == i][data_points["pilot"].isin(teams_pilot[team])]["points"].sum())
        data_team_points["cum_points"].append(data_points[data_points["index"] == i][data_points["pilot"].isin(teams_pilot[team])]["cum_points"].sum())
        data_team_points["index"].append(i)
        data_team_points["circuit_name"].append(data_points[data_points["index"] == i]["circuit_name"].unique()[0])
data_team_points = pd.DataFrame(data_team_points)

# last gp
last_gp = data_all.groupby("pilot").tail(1)

# colors
colors = json.loads(open("colors.json").read())

# lap of last gp
lap_df = pd.read_json("lap_data.json")

# average lap time of each pilot on last gp
dico = {pilot: lap_df.groupby("pilot").mean()["time"][pilot] for pilot in lap_df["pilot"].unique()}
last_gp["average_lap_time"] = last_gp["pilot"].map(dico)

# Extract the battles between pilots
with open("battles.json") as jsonfile:
    battles = json.load(jsonfile)
pilot_battles = battles["battles"]["pilots"]

df_battles_pilot = []  # List of battles dataframes
for base_pilot, battle in pilot_battles.items():
    # Filter data with pilots of the battle
    df_bt = data_points[data_points["pilot"].isin(battle)]
    
    # Compute the difference of points
    df_bt["diff"] = df_bt["cum_points"] - df_bt[df_bt["pilot"] == base_pilot]["cum_points"].values[df_bt["index"]]
    df_battles_pilot.append(df_bt)

# Extract the battles between teams
team_battles = battles["battles"]["teams"]

df_battles_team = []  # List of battles dataframes
for base_team, battle  in team_battles.items():
    # Filter data with teams of the battle
    df_bt = data_team_points[data_team_points["team"].isin(battle)]
    
    # Compute the difference of points
    df_bt["diff"] = df_bt["cum_points"] - df_bt[df_bt["team"] == base_team]["cum_points"].values[df_bt["index"]]
    df_battles_team.append(df_bt)

# initialize the barplots of the battles
pilot_battle_compare = {
    "pilot": list(pilot_battles.values())[0],
    "podium": [0 for i in range(len(list(pilot_battles.values())[0]))],
    "win": [0 for i in range(len(list(pilot_battles.values())[0]))],
    "pole": [0 for i in range(len(list(pilot_battles.values())[0]))]
}

team_battle_compare = {
    "team": list(team_battles.values())[0],
    "podium": [0 for i in range(len(list(team_battles.values())[0]))],
    "win": [0 for i in range(len(list(team_battles.values())[0]))],
    "pole": [0 for i in range(len(list(team_battles.values())[0]))]
}

# compute the number of podiums, wins and poles of each pilot
for j in range(len(pilot_battle_compare["pilot"])):
    pilot = pilot_battle_compare["pilot"][j]
    pilot_battle_compare["podium"][j] = data_all[data_all["pilot"] == pilot][data_all["result"] < 4].shape[0]
    pilot_battle_compare["win"][j] = data_all[data_all["pilot"] == pilot][data_all["result"] == 1].shape[0]
    pilot_battle_compare["pole"][j] = data_all[data_all["pilot"] == pilot][data_all["grid"] == 1].shape[0]
for j in range(len(team_battle_compare["team"])):
    team = team_battle_compare["team"][j]
    team_battle_compare["podium"][j] = data_all[data_all["pilot"].isin(teams_pilot[team])][data_all["result"] < 4].shape[0]
    team_battle_compare["win"][j] = data_all[data_all["pilot"].isin(teams_pilot[team])][data_all["result"] == 1].shape[0]
    team_battle_compare["pole"][j] = data_all[data_all["pilot"].isin(teams_pilot[team])][data_all["grid"] == 1].shape[0]
pilot_battle_compare = pd.DataFrame(pilot_battle_compare)
team_battle_compare = pd.DataFrame(team_battle_compare)
df_test = pd.DataFrame({
        "pilot": 3*list(pilot_battle_compare["pilot"]),
        "type": ["podium"]*len(pilot_battle_compare["pilot"])+["win"]*len(pilot_battle_compare["pilot"])+["pole"]*len(pilot_battle_compare["pilot"]),
        "value": list(pilot_battle_compare["podium"]) + list(pilot_battle_compare["win"]) + list(pilot_battle_compare["pole"])
        })

# diverging bar chart between lec and ham
# pilot choice
p1 = widgets.Dropdown(
    options=list(data_all["pilot"].unique()),
    value='lec',
    description='Left Pilot:',
    style={'background-color': 'black', 'color': 'white'}
)
p2 = widgets.Dropdown(
    options=list(data_all["pilot"].unique()),
    value='ham',
    description='Right Pilot:',
)
df = None
pilot1 = p1.value
pilot2 = p2.value

def compare_pilots():
    global df, pilot1, pilot2
    pilot1 = p1.value
    pilot2 = p2.value
    
    # initialize the dataframe
    names = ["Pilots", "Better Start", "Podiums", "Pole Position", "Better result", "Wins", "Improving"]
    data1 = data_all[data_all["pilot"] == pilot1]
    data2 = data_all[data_all["pilot"] == pilot2]
    df = pd.DataFrame(columns=["category", "pilot 1", "pilot 2"])

    # fill the dataframe
    for i in range(len(names)):
        df.loc[i] = [names[i], 0, 0]
    for i in range(data1.shape[0]):
        # better start
        if data1["grid"].iloc[i] < data2["grid"].iloc[i]:
            df["pilot 1"][df["category"] == "Better Start"] += 1
        else:
            df["pilot 2"][df["category"] == "Better Start"] += 1
        
        # podiums
        if data1["result"].iloc[i] <= 3:
            df["pilot 1"][df["category"] == "Podiums"] += 1
        if data2["result"].iloc[i] <= 3:
            df["pilot 2"][df["category"] == "Podiums"] += 1
        
        # pole position
        if data1["grid"].iloc[i] == 1:
            df["pilot 1"][df["category"] == "Pole Position"] += 1
        elif data2["grid"].iloc[i] == 1:
            df["pilot 2"][df["category"] == "Pole Position"] += 1
        
        # better result
        if data1["result"].iloc[i] < data2["result"].iloc[i]:
            df["pilot 1"][df["category"] == "Better result"] += 1
        else:
            df["pilot 2"][df["category"] == "Better result"] += 1
        
        # wins
        if data1["result"].iloc[i] == 1:
            df["pilot 1"][df["category"] == "Wins"] += 1
        elif data2["result"].iloc[i] == 1:
            df["pilot 2"][df["category"] == "Wins"] += 1
        
        # improving
        if data1["result"].iloc[i] < data1["grid"].iloc[i]:
            df["pilot 1"][df["category"] == "Improving"] += 1
        if data2["result"].iloc[i] < data2["grid"].iloc[i]:
            df["pilot 2"][df["category"] == "Improving"] += 1

compare_pilots()

# plot
fig = make_subplots(rows=1, cols=2, subplot_titles=(pilot1, pilot2), shared_yaxes=True)
fig.add_trace(go.Bar(x=df["pilot 1"], y=df["category"], orientation="h", marker_color=colors["pilot"][pilot1]), row=1, col=1)
fig.update_xaxes(title_text=pilot1, row=1, col=1, autorange="reversed")
fig.add_trace(go.Bar(x=df["pilot 2"], y=df["category"], orientation="h", marker_color=colors["pilot"][pilot2]), row=1, col=2)
fig.update_xaxes(title_text=pilot2, row=1, col=2)
fig.update_layout(template="plotly_dark")

def update(pilot1, pilot2):
    compare_pilots()
    plots.data[0].x = df["pilot 1"]
    plots.data[0].marker.color = colors["pilot"][pilot1]
    plots.layout.annotations[0].text = pilot1
    
    plots.data[1].x = df["pilot 2"]
    plots.data[1].marker.color = colors["pilot"][pilot2]
    plots.layout.annotations[1].text = pilot2

p1.observe(lambda x: update(p1.value, p2.value), names="value")
p2.observe(lambda x: update(p1.value, p2.value), names="value")

plots = go.FigureWidget(fig)

# dnf count per pilot and per team (1 = dnf, 0 = not dnf)
pilot_dnf_count = data_all[data_all["dnf"] == 1]["pilot"].value_counts()
pilot_dnf_count = pd.DataFrame({"pilot": list(pilot_dnf_count.index), "dnf": list(pilot_dnf_count.values)})
team_dnf_count = data_all[data_all["dnf"] == 1]["team"].value_counts()
team_dnf_count = pd.DataFrame({"team": list(team_dnf_count.index), "dnf": list(team_dnf_count.values)})

# circuit data
circuit_df = pd.read_json("circuit_2023.json")
circuit_df["lap length"] = circuit_df["lap nb"] * circuit_df["length"]
circuit_df["ratio"] = circuit_df["lap length"] / circuit_df["turn nb"]

C:\Users\Shayane\AppData\Local\Temp\ipykernel_24608\1414089624.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_points["cum_points"] = data_points.groupby("pilot").cumsum()
C:\Users\Shayane\AppData\Local\Temp\ipykernel_24608\1414089624.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_team_points["points"].append(data_points[data_points["index"] == i][data_points["pilot"].isin(teams_pilot[team])]["points"].sum())
C:\Users\Shayane\AppData\Local\Temp\ipykernel_24608\1414089624.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_team_points["cum_points"].append(data_points[data_points["index"] == i][data_points["pilot"].isin(teams_pilot[team])]["cum_points"].sum())
C:\Users\Shayane\AppData\Local\Temp\ipykernel_24

### Frontend

#### Last GP

In [3]:
px.scatter(last_gp, x="pilot", y="best_lap", title="Time of fastest lap of each pilot (s)", marginal_y="histogram", template="plotly_dark")

In [4]:
px.scatter(last_gp, x="pilot", y="average_lap_time", title="Average lap time of each pilot (s)", marginal_y="histogram", template="plotly_dark")

In [5]:
px.scatter(last_gp, x="pilot", y="avg_speed", title="Mean speed of the fastes_lap of each pilot (kmh)", marginal_y="histogram", template="plotly_dark")

In [6]:
px.scatter(last_gp, x="pilot", y="time", title="Total time (s)", marginal_y="histogram", template="plotly_dark")
# no data displayed if the pilot didn't finish or had one lap of delay

In [7]:
# px.scatter(last_gp, x="pilot", y="improvement", title="Improvement of the pilots", marginal_y="histogram", template="plotly_dark")
px.bar(last_gp, x="pilot", y="improvement", title="Improvement of the pilots", color="pilot", color_discrete_map=colors["pilot"], template="plotly_dark")
# invert the y axis

In [8]:
fig = go.Figure(layout=go.Layout(title="Total time (s)", template="plotly_dark"))
fig.add_trace(go.Box(y=last_gp["best_lap"], name="boxplot fastest_lap time", boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=last_gp["pilot"]))

In [9]:
fig = go.Figure(layout=go.Layout(title="Mean speed of the fastes_lap of each pilot (kmh)", template="plotly_dark"))
fig.add_trace(go.Box(y=last_gp["avg_speed"], name="boxplot fastest_lap time", boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=last_gp["pilot"]))

In [10]:
fig = go.Figure(layout=go.Layout(title="Total time of race of each pilot (s)", template="plotly_dark"))
fig.add_trace(go.Box(y=last_gp["time"], name="boxplot total time", boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=last_gp["pilot"]))

In [11]:
fig = go.Figure(layout=go.Layout(title="Improvement of each pilot (s)", template="plotly_dark"))
fig.add_trace(go.Box(y=last_gp["improvement"], name="boxplot improvement", boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=last_gp["pilot"]))

##### Laps

In [12]:
fig = go.Figure(layout=go.Layout(title="Lap time of each pilot", template="plotly_dark"))
for pilot in lap_df["pilot"].unique():
    fig.add_traces(go.Box(y=lap_df["time"][lap_df["pilot"]==pilot], name=pilot, fillcolor=colors["pilot"][pilot], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=lap_df["lap"]))
# fig.add_traces(go.Box(x=lap_df["pilot"], y=lap_df["time"], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))
fig.show()

In [12]:
px.line(lap_df, x="lap", y="time", color="pilot", color_discrete_map=colors["pilot"], title="Lap time evolution along the race", markers=True, template="plotly_dark")

In [13]:
fig = px.line(lap_df, x="lap", y="position", color="pilot", color_discrete_map=colors["pilot"], title="Pilot's position along the race", markers=True, template="plotly_dark")
fig.update_layout(yaxis={'autorange': 'reversed'})

In [14]:
px.histogram(lap_df, x='time', title='Lap Time Histogram', template="plotly_dark")

In [15]:
px.density_heatmap(lap_df, x="pilot", y="position", nbinsx=20, nbinsy=20, title="Postion of each pilot during the race", template="plotly_dark")

## General

In [16]:
px.line(data_points, x="index", y="cum_points", color="pilot", color_discrete_map=colors["pilot"], title="Pilots cumlated points", template="plotly_dark", markers=True)

In [17]:
px.line(data_team_points, x="index", y="cum_points", color="team", color_discrete_map=colors["team"], title="Teams cumulated points", template="plotly_dark", markers=True)

In [18]:
px.density_heatmap(data_all, x="pilot", y="grid", title="Pilot's grid position", nbinsx=20, nbinsy=20, template="plotly_dark")

In [19]:
px.density_heatmap(data_all, x="pilot", y="result", nbinsx=20, nbinsy=20, title="Pilot's result position", template="plotly_dark")

In [20]:
px.density_heatmap(data_all, x="pilot", y="dnf", nbinsx=20, nbinsy=2, title="Pilot's DNF", template="plotly_dark")

## Pilot battle Part

Here I am interested on the battles between pilots

In [21]:
px.line(df_battles_pilot[0], x="circuit_name", y="diff", color="pilot", color_discrete_map=colors["pilot"], title="Top Pilot battle", markers=True, template="plotly_dark")

In [22]:
px.line(df_battles_pilot[1], x="circuit_name", y="diff", color="pilot", color_discrete_map=colors["pilot"], title="Middle Pilot battle", markers=True, template="plotly_dark")

In [23]:
px.scatter(pilot_dnf_count, x="pilot", y="dnf", color="pilot", color_discrete_map=colors["pilot"], title="Number of dnf per pilot", marginal_y="histogram", template="plotly_dark")

## Teams battle Part

Here I am interested on the battles between teams

In [24]:
px.line(df_battles_team[0], x="circuit_name", y="diff", color="team", color_discrete_map=colors["team"], title="Top Team battle", markers=True, template="plotly_dark")

In [25]:
px.line(df_battles_team[1], x="circuit_name", y="diff", color="team", color_discrete_map=colors["team"], title="Bottom Team battle", markers=True, template="plotly_dark")

In [26]:
px.scatter(team_dnf_count, x="team", y="dnf", color="team", color_discrete_map=colors["team"], title="Number of dnf per team", marginal_y="histogram", template="plotly_dark")

#### Battle Compare

In [27]:
fig = make_subplots(rows=3, cols=2, subplot_titles=("Top Pilot podiums", "Top Team podiums", "Top Pilot wins", "Top Team wins", "Top Pilot poles", "Top Team poles"))

fig.add_trace(go.Bar(x=pilot_battle_compare["pilot"], y=pilot_battle_compare["podium"], name="Podiums"), row=1, col=1)
fig.add_trace(go.Bar(x=pilot_battle_compare["pilot"], y=pilot_battle_compare["win"], name="Wins"), row=2, col=1)
fig.add_trace(go.Bar(x=pilot_battle_compare["pilot"], y=pilot_battle_compare["pole"], name="Poles"), row=3, col=1)

fig.add_trace(go.Bar(x=team_battle_compare["team"], y=team_battle_compare["podium"], name="Podiums"), row=1, col=2)
fig.add_trace(go.Bar(x=team_battle_compare["team"], y=team_battle_compare["win"], name="Wins"), row=2, col=2)
fig.add_trace(go.Bar(x=team_battle_compare["team"], y=team_battle_compare["pole"], name="Poles"), row=3, col=2)

fig.update_layout(title="Top Pilot battle", template="plotly_dark")

fig.show()


In [28]:
px.line_polar(df_test, r="value", theta="type", color="pilot", color_discrete_map=colors["pilot"], template="plotly_dark", line_close=True)

c:\Users\Shayane\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\Shayane\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\Shayane\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\Shayane\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\Shayane\AppData\Local\Programs\Pyth

In [29]:
container = widgets.HBox([p1, p2], layout=widgets.Layout(justify_content="center"))
widgets.VBox([container, plots])

### Per pilot

In [30]:
fig = go.Figure(layout=go.Layout(title="Pilot's results", template="plotly_dark"))
for pilot in data_all["pilot"].unique():
    fig.add_traces(go.Box(y=data_all["result"][data_all["pilot"]==pilot], name=pilot, fillcolor=colors["pilot"][pilot], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))
fig.show()

In [31]:
fig = go.Figure(layout=go.Layout(title="Pilot's grid", template="plotly_dark"))
for pilot in data_all["pilot"].unique():
    fig.add_traces(go.Box(y=data_all["grid"][data_all["pilot"]==pilot], name=pilot, fillcolor=colors["pilot"][pilot], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))
fig.show()

In [32]:
fig = go.Figure(layout=go.Layout(title="Position improvement from grid to result", template="plotly_dark"))
for pilot in data_all["pilot"].unique():
    fig.add_trace(go.Box(y=data_all["improvement"][data_all["pilot"] == pilot], name=pilot, boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=data_all["circuit_name"], fillcolor=colors["pilot"][pilot]))
fig.show()

### global

#### results

In [33]:
# imshow du result avec le pilot en x et le circuit en y
px.imshow(data_all.pivot("pilot", "circuit_name", "result"), title="Pilot's results", text_auto=True, template="plotly_dark")

C:\Users\Shayane\AppData\Local\Temp\ipykernel_24608\999611635.py:2: FutureWarning:

In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.



In [34]:
fig = px.line(data_all, x="circuit_name", y="result", color="pilot", color_discrete_map=colors["pilot"], title="Pilot's evolution of results", markers=True, template="plotly_dark")
fig.update_layout(yaxis={'autorange': 'reversed'})

#### Grid

In [35]:
# imshow du grid avec le pilot en x et le circuit en y
px.imshow(data_all.pivot("pilot", "circuit_name", "grid"), title="Pilot's grids", text_auto=True, template="plotly_dark")

C:\Users\Shayane\AppData\Local\Temp\ipykernel_24608\3253442934.py:2: FutureWarning:

In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.



In [36]:
fig = px.line(data_all, x="circuit_name", y="grid", color="pilot", color_discrete_map=colors["pilot"], title="Pilot's evolution of grids", markers=True, template="plotly_dark")
fig.update_layout(yaxis={'autorange': 'reversed'})

## Circuit data

In [37]:
fig = go.Figure(layout=go.Layout(title="Circuit average speed on fastest laps", template="plotly_dark"))
fig.add_traces(go.Box(x=data_all["circuit_name"], y=data_all["avg_speed"], text=data_all["pilot"], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))

In [38]:
fig = go.Figure(layout=go.Layout(title="Circuit best lap time", template="plotly_dark"))
fig.add_traces(go.Box(x=data_all["circuit_name"], y=data_all["best_lap"], text=data_all["pilot"], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))

In [39]:
fig = go.Figure(layout=go.Layout(title="Circuit Length", template="plotly_dark"))
fig.add_trace(go.Box(y=circuit_df["length"], name="boxplot circuit length", boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=circuit_df["name"]))

In [40]:
fig = go.Figure(layout=go.Layout(title="Circuit Lap Number", template="plotly_dark"))
fig.add_traces(go.Box(y=circuit_df["lap nb"], name="boxplot number of lap", boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))

In [41]:
px.scatter_geo(circuit_df, lat="lat", lon="long", hover_name="name", title="Circuit Location", template="plotly_dark")

In [42]:
fig = go.Figure(layout=go.Layout(title="Lap number vs length", template="plotly_dark"))
fig.add_traces(go.Scatter(x=circuit_df["lap nb"], y=circuit_df["length"], mode="markers", text=circuit_df["name"], name="circuit data"))
fig.add_traces(go.Line(x=[i for i in range(30, 80)], y=[300000/i for i in range(30, 80)], name="expected tendancy"))

c:\Users\Shayane\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [43]:
fig = go.Figure(layout=go.Layout(title="distance between turns vs dnf number", template="plotly_dark"))
fig.add_traces(go.Scatter(x=circuit_df["lap nb"], y=data_all.groupby("circuit_name")["dnf"].sum().reset_index()["dnf"], text=circuit_df["name"], mode="markers", name="circuit data"))

In [44]:
fig = go.Figure(layout=go.Layout(title="distance between turns vs avg_speed", template="plotly_dark"))
fig.add_traces(go.Scatter(x=circuit_df["lap nb"], y=data_all.groupby("circuit_name")["avg_speed"].mean().reset_index()["avg_speed"], text=circuit_df["name"], mode="markers", name="circuit data"))

In [45]:
fig = go.Figure(layout=go.Layout(title="distance between turns vs best lap time", template="plotly_dark"))
fig.add_traces(go.Scatter(x=circuit_df["lap nb"], y=data_all.groupby("circuit_name")["best_lap"].mean().reset_index()["best_lap"], text=circuit_df["name"], mode="markers", name="circuit data"))

In [46]:
fig = make_subplots(rows=3, cols=1, subplot_titles=("dnf number", "average speed", "best lap time"), shared_xaxes=True, vertical_spacing=0.1, horizontal_spacing=0.1)

fig.add_trace(go.Scatter(x=circuit_df["lap nb"], y=data_all.groupby("circuit_name")["dnf"].sum().reset_index()["dnf"], mode="markers", text=circuit_df["name"], name="circuit data"), row=1, col=1)
fig.add_trace(go.Scatter(x=circuit_df["lap nb"], y=data_all.groupby("circuit_name")["avg_speed"].mean().reset_index()["avg_speed"], text=circuit_df["name"], mode="markers", name="circuit data"), row=2, col=1)
fig.add_trace(go.Scatter(x=circuit_df["lap nb"], y=data_all.groupby("circuit_name")["best_lap"].mean().reset_index()["best_lap"], text=circuit_df["name"], mode="markers", name="circuit data"), row=3, col=1)

fig.update_layout(title="Top Pilot battle", template="plotly_dark")

fig.show()